In [2]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt

In [3]:
def load_data_from_csv(data_path):
    return pd.read_csv(data_path)

In [5]:
MAIN_DIR_PATH = '../01 PM2.5 Chinese Weather data'
#PREP_MAIN_DIR_PATH = '../Prep_FiveCitiePMData_MEAN'
cities_data_path_list = os.listdir(MAIN_DIR_PATH)

In [9]:
print(*cities_data_path_list, sep='\n')

BeijingPM20100101_20151231.csv
ChengduPM20100101_20151231.csv
GuangzhouPM20100101_20151231.csv
ShanghaiPM20100101_20151231.csv
ShenyangPM20100101_20151231.csv


Beijing city data analysis

In [10]:
sample_data_path = os.path.join(MAIN_DIR_PATH, cities_data_path_list[0])
print(sample_data_path)
data = load_data_from_csv(sample_data_path)

../01 PM2.5 Chinese Weather data\BeijingPM20100101_20151231.csv


1. DATA ANALYSIS

1.1 Look at distribution and find outliers

In [11]:
# Here are the features we have
for idx, col in enumerate(data.columns):
    print(idx + 1, col)

1 No
2 year
3 month
4 day
5 hour
6 season
7 PM_Dongsi
8 PM_Dongsihuan
9 PM_Nongzhanguan
10 PM_US Post
11 DEWP
12 HUMI
13 PRES
14 TEMP
15 cbwd
16 Iws
17 precipitation
18 Iprec
